In [19]:
import pandas as pd
from gensim.models import HdpModel
from gensim import corpora
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [20]:
# Baca data
df = pd.read_csv('../data/dataHasilPreprocessing/dataPreprocessing.csv')
df['Sentimen'] = df['Sentimen'].map({'Positif': 1, 'Negatif': 0})
df

,Ulasan,Sentimen
0,ulas,0
1,NaN,0
2,bahan aja warna navy nya beda,0
3,kasih catat order warna kuning navy baca kirim...,0
4,kecil,0
...,...,...
990,barang selamat terima kasih bahan celana nya s...,0
991,kecil,0
992,pesan cuna barang rusak coba konfirmasi tanggap,0
993,karet pinggang nya kencang,0


In [21]:
# Hapus duplikat dan nilai yang hilang
df = df.drop_duplicates(subset=['Ulasan'])
df = df.dropna()

In [22]:
tokenized_reviews = [review.lower().split() for review in df['Ulasan']]
dictionary = corpora.Dictionary(tokenized_reviews)
corpus = [dictionary.doc2bow(review) for review in tokenized_reviews]

In [23]:
hdp_model = HdpModel(corpus, dictionary)
num_topics = 5  # Misalnya, kita akan menggunakan 5 topik

In [24]:
# Mendapatkan distribusi topik untuk setiap ulasan
topic_distributions = [hdp_model[doc] for doc in corpus]

# Mendapatkan bobot topik untuk setiap ulasan
aspect_list = []
for dist in topic_distributions:
    aspect = max(dist, key=lambda x: x[1])[0]  # Ambil topik dengan bobot terbesar
    aspect_list.append(aspect)

In [25]:
# Tambahkan atribut aspek ke dalam dataframe
df['Aspect'] = aspect_list

In [26]:
# Model BERT
model_name = 'indobenchmark/indobert-base-p1'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# Pra-pemrosesan data untuk BERT
reviews = df['Ulasan'].tolist()
labels = df['Sentimen'].tolist()

In [28]:
max_length = 128
input_ids = []
attention_masks = []

# Tokenisasi data
for review in reviews:
    encoded_dict = tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=max_length,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\transformers\tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [29]:
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)
labels = tf.convert_to_tensor(labels)

In [30]:
# Split data menjadi train dan test sets
train_indices, test_indices = train_test_split(range(len(input_ids)), test_size=0.2, random_state=42)
train_indices = tf.convert_to_tensor(train_indices, dtype=tf.int32)
test_indices = tf.convert_to_tensor(test_indices, dtype=tf.int32)

train_input_ids = tf.gather(input_ids, train_indices)
train_attention_masks = tf.gather(attention_masks, train_indices)
train_labels = tf.gather(labels, train_indices)

test_input_ids = tf.gather(input_ids, test_indices)
test_attention_masks = tf.gather(attention_masks, test_indices)
test_labels = tf.gather(labels, test_indices)

In [31]:
# Konfigurasi pelatihan
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [32]:
# Latih model
batch_size = 16
epochs = 15

history = model.fit(
    [train_input_ids, train_attention_masks],
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([test_input_ids, test_attention_masks], test_labels)
)

Epoch 1/15
48/48 [==============================] - 22s 256ms/step - loss: 0.4959 - accuracy: 0.8399 - val_loss: 0.3688 - val_accuracy: 0.8789
Epoch 2/15
48/48 [==============================] - 11s 220ms/step - loss: 0.2676 - accuracy: 0.9021 - val_loss: 0.2907 - val_accuracy: 0.8842
Epoch 3/15
48/48 [==============================] - 11s 222ms/step - loss: 0.1453 - accuracy: 0.9484 - val_loss: 0.3349 - val_accuracy: 0.8526
Epoch 4/15
48/48 [==============================] - 11s 220ms/step - loss: 0.0809 - accuracy: 0.9775 - val_loss: 0.4428 - val_accuracy: 0.8684
Epoch 5/15
48/48 [==============================] - 11s 221ms/step - loss: 0.0423 - accuracy: 0.9947 - val_loss: 0.5316 - val_accuracy: 0.8684
Epoch 6/15
48/48 [==============================] - 11s 223ms/step - loss: 0.0265 - accuracy: 0.9947 - val_loss: 0.5777 - val_accuracy: 0.8368
Epoch 7/15
48/48 [==============================] - 11s 221ms/step - loss: 0.0312 - accuracy: 0.9934 - val_loss: 0.5688 - val_accuracy: 0.8737

In [33]:
# Evaluasi model
model.evaluate([test_input_ids, test_attention_masks], test_labels)

6/6 [==============================] - 1s 133ms/step - loss: 0.6604 - accuracy: 0.8684


[0.6604195237159729, 0.8684210777282715]

In [34]:
# Tampilkan aspek dalam dataframe
df_aspect = pd.DataFrame({'Aspect': aspect_list, 'Sentimen': df['Sentimen'], 'Ulasan': df['Ulasan']})
df_aspect

,Aspect,Sentimen,Ulasan
0,78,0,ulas
2,41,0,bahan aja warna navy nya beda
3,82,0,kasih catat order warna kuning navy baca kirim...
4,4,0,kecil
5,71,0,tau layan
...,...,...,...
989,115,0,cocok
990,71,0,barang selamat terima kasih bahan celana nya s...
992,20,0,pesan cuna barang rusak coba konfirmasi tanggap
993,139,0,karet pinggang nya kencang


In [35]:
# Evaluasi model sebelum menggunakan HDP
model.evaluate([test_input_ids, test_attention_masks], test_labels)

6/6 [==============================] - 1s 133ms/step - loss: 0.6604 - accuracy: 0.8684


[0.6604195237159729, 0.8684210777282715]

In [36]:
# Buat prediksi untuk data sebelum menggunakan HDP
test_predictions_before_hdp = model.predict([test_input_ids, test_attention_masks])
predicted_labels_before_hdp = tf.argmax(test_predictions_before_hdp.logits, axis=1)

# Tampilkan laporan klasifikasi sebelum menggunakan HDP
classification_rep_before_hdp = classification_report(test_labels, predicted_labels_before_hdp)
print("Classification Report Sebelum Menggunakan HDP:")
print(classification_rep_before_hdp)

6/6 [==============================] - 3s 134ms/step
Classification Report Sebelum Menggunakan HDP:
              precision    recall  f1-score   support

           0       0.57      0.55      0.56        29
           1       0.92      0.93      0.92       161

    accuracy                           0.87       190
   macro avg       0.75      0.74      0.74       190
weighted avg       0.87      0.87      0.87       190



In [37]:
# Latih model setelah menggunakan HDP
train_input_ids_after_hdp = train_input_ids
train_attention_masks_after_hdp = train_attention_masks
train_labels_after_hdp = train_labels

test_input_ids_after_hdp = test_input_ids
test_attention_masks_after_hdp = test_attention_masks
test_labels_after_hdp = test_labels

In [38]:
# Latih model
history_after_hdp = model.fit(
    [train_input_ids_after_hdp, train_attention_masks_after_hdp],
    train_labels_after_hdp,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([test_input_ids_after_hdp, test_attention_masks_after_hdp], test_labels_after_hdp)
)

Epoch 1/15
48/48 [==============================] - 11s 224ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.7041 - val_accuracy: 0.8684
Epoch 2/15
48/48 [==============================] - 11s 224ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.7158 - val_accuracy: 0.8684
Epoch 3/15
48/48 [==============================] - 11s 222ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7273 - val_accuracy: 0.8684
Epoch 4/15
48/48 [==============================] - 11s 221ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7725 - val_accuracy: 0.8632
Epoch 5/15
48/48 [==============================] - 11s 220ms/step - loss: 9.2317e-04 - accuracy: 1.0000 - val_loss: 0.7851 - val_accuracy: 0.8579
Epoch 6/15
48/48 [==============================] - 11s 221ms/step - loss: 8.4998e-04 - accuracy: 1.0000 - val_loss: 0.7865 - val_accuracy: 0.8684
Epoch 7/15
48/48 [==============================] - 11s 220ms/step - loss: 7.7885e-04 - accuracy: 1.0000 - val_loss: 0.7930 - val_accu

In [39]:
# Evaluasi model setelah menggunakan HDP
model.evaluate([test_input_ids_after_hdp, test_attention_masks_after_hdp], test_labels_after_hdp)

6/6 [==============================] - 1s 136ms/step - loss: 0.8600 - accuracy: 0.8684


[0.8599599003791809, 0.8684210777282715]

In [40]:
# Buat prediksi untuk data setelah menggunakan HDP
test_predictions_after_hdp = model.predict([test_input_ids_after_hdp, test_attention_masks_after_hdp])
predicted_labels_after_hdp = tf.argmax(test_predictions_after_hdp.logits, axis=1)

# Tampilkan laporan klasifikasi setelah menggunakan HDP
classification_rep_after_hdp = classification_report(test_labels_after_hdp, predicted_labels_after_hdp)
print("Classification Report Sesudah Menggunakan HDP:")
print(classification_rep_after_hdp)

6/6 [==============================] - 1s 134ms/step
Classification Report Sesudah Menggunakan HDP:
              precision    recall  f1-score   support

           0       0.57      0.55      0.56        29
           1       0.92      0.93      0.92       161

    accuracy                           0.87       190
   macro avg       0.75      0.74      0.74       190
weighted avg       0.87      0.87      0.87       190

